In [ ]:
import os
import json
import shutil
from tqdm import tqdm

# Path ke file COCO JSON dan folder output YOLO
coco_json_path = "../dataset/Trash-COCO-Subset/test/_annotations.coco.json"
image_folder = "../dataset/Trash-COCO-Subset/test/"
output_folder = "../dataset/Trash-Yolo-Subset/test/"

# Membuat subfolder "images" dan "labels" dalam folder output
images_output_folder = os.path.join(output_folder, "images")
labels_output_folder = os.path.join(output_folder, "labels")
os.makedirs(images_output_folder, exist_ok=True)
os.makedirs(labels_output_folder, exist_ok=True)

# Load data COCO
with open(coco_json_path, 'r') as file:
    coco_data = json.load(file)

# Mapping kategori COCO ke YOLO class ID
category_mapping = {cat["id"]: idx for idx, cat in enumerate(coco_data["categories"])}

# Fungsi untuk konversi bbox COCO ke YOLO
def coco_bbox_to_yolo_bbox(bbox, img_width, img_height):
    x, y, w, h = bbox
    x_center = (x + w / 2) / img_width
    y_center = (y + h / 2) / img_height
    w = w / img_width
    h = h / img_height
    return x_center, y_center, w, h

# Proses setiap gambar
for image_info in tqdm(coco_data["images"], desc="Converting COCO to YOLO"):
    image_id = image_info["id"]
    file_name = image_info["file_name"]
    img_width = image_info["width"]
    img_height = image_info["height"]
    
    # File output YOLO untuk gambar ini
    label_file_path = os.path.join(labels_output_folder, os.path.splitext(file_name)[0] + ".txt")
    
    # Ambil semua anotasi untuk gambar ini
    annotations = [
        ann for ann in coco_data["annotations"] if ann["image_id"] == image_id
    ]
    
    # Tulis anotasi ke file YOLO
    with open(label_file_path, 'w') as txt_file:
        for ann in annotations:
            category_id = ann["category_id"]
            yolo_class_id = category_mapping[category_id]
            bbox = ann["bbox"]
            
            # Konversi bbox ke format YOLO
            yolo_bbox = coco_bbox_to_yolo_bbox(bbox, img_width, img_height)
            
            # Tulis ke file
            txt_file.write(f"{yolo_class_id} {' '.join(map(str, yolo_bbox))}\n")
    
    # Salin gambar ke folder "images"
    source_image_path = os.path.join(image_folder, file_name)
    destination_image_path = os.path.join(images_output_folder, file_name)
    if os.path.exists(source_image_path):
        shutil.copy(source_image_path, destination_image_path)

print("Konversi selesai! Dataset YOLO tersimpan di:", output_folder)


Converting COCO to YOLO: 100%|██████████| 110/110 [00:00<00:00, 1762.51it/s]

Konversi selesai! Dataset YOLO tersimpan di: ../dataset/Trash-Yolo-Subset/test/


In [7]:
import os
import json
from PIL import Image

# Path dataset COCO
coco_json_path = "../dataset/Trash-COCO-Subset/test/_annotations.coco.json"
images_folder = "../dataset/Trash-COCO-Subset/test/"
output_folder = "../dataset/Trash/test/"

# Membuat folder output jika belum ada
os.makedirs(output_folder, exist_ok=True)

# Load file JSON COCO
with open(coco_json_path, 'r') as file:
    coco_data = json.load(file)

# Mapping kategori (category_id -> category_name)
category_mapping = {cat['id']: cat['name'] for cat in coco_data['categories']}

# Menambahkan padding (50% dari ukuran bounding box)
padding_factor = 1

# Memproses setiap anotasi
for annotation in coco_data['annotations']:
    # Informasi dasar
    image_id = annotation['image_id']
    category_id = annotation['category_id']
    category_name = category_mapping[category_id]
    bbox = annotation['bbox']  # Format: [x, y, width, height]

    # Cari file gambar
    image_info = next(img for img in coco_data['images'] if img['id'] == image_id)
    image_path = os.path.join(images_folder, image_info['file_name'])

    # Buka gambar dan tambahkan padding pada bbox
    with Image.open(image_path) as img:
        img_width, img_height = img.size
        x, y, w, h = map(int, bbox)

        # Hitung padding
        pad_w = int(w * padding_factor)
        pad_h = int(h * padding_factor)

        # Hitung bounding box dengan padding
        x_min = max(0, x - pad_w)
        y_min = max(0, y - pad_h)
        x_max = min(img_width, x + w + pad_w)
        y_max = min(img_height, y + h + pad_h)

        # Crop gambar dengan bounding box yang diperluas
        cropped_img = img.crop((x_min, y_min, x_max, y_max))

        # Buat folder kategori jika belum ada
        category_folder = os.path.join(output_folder, category_name)
        os.makedirs(category_folder, exist_ok=True)

        # Simpan gambar hasil crop
        output_image_path = os.path.join(
            category_folder, f"{image_info['file_name'].split('.')[0]}_{annotation['id']}.jpg"
        )
        cropped_img.save(output_image_path)

print("Proses selesai! Dataset telah diproses ke dalam struktur folder berdasarkan kategori.")


Proses selesai! Dataset telah diproses ke dalam struktur folder berdasarkan kategori.
